Query formulation

In [1]:
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
import nltk
import numpy as np
#nltk.download()

In [2]:
tokens = nltk.word_tokenize("What are the different OS for personal computers?")
print(nltk.pos_tag(tokens))
tagged = nltk.pos_tag(tokens)

[('What', 'WP'), ('are', 'VBP'), ('the', 'DT'), ('different', 'JJ'), ('OS', 'NNP'), ('for', 'IN'), ('personal', 'JJ'), ('computers', 'NNS'), ('?', '.')]


In [3]:
query_words_tuple = list(filter(lambda x: 'NN' in x[1], tagged))
query_words = [key for key, _ in query_words_tuple]
query_words

['OS', 'computers']

Document retrieval

In [4]:
import os
basepath = '.'
for fname in os.listdir(basepath):
    path = os.path.join(basepath, fname)
    if os.path.isdir(path):
        for query_word in query_words:
            if fname == query_word:
                document = path
                print(path)

.\OS


Passage retrieval

In [5]:
#!pip install rank-bm25
from rank_bm25 import BM25Okapi

corpus = []
for filename in os.listdir(document):
    print(filename)
    with open(os.path.join(document, filename), 'r') as file:
        corpus.append(file.read())

corpus

OS_extended.txt
OS_memory.txt
OS_program_execution.txt


["An operating system is the most important software that runs on a computer. It manages the computer's memory and processes, as well as all of its software and hardware. It also allows you to communicate with the computer without knowing how to speak the computer's language. Without an operating system, a computer is useless.\nYour computer's operating system (OS) manages all of the software and hardware on the computer. Most of the time, there are several different computer programs running at the same time, and they all need to access your computer's central processing unit (CPU), memory, and storage. The operating system coordinates all of this to make sure each program gets what it needs.\nOperating systems usually come pre-loaded on any computer you buy. Most people use the operating system that comes with their computer, but it's possible to upgrade or even change operating systems. The three most common operating systems for personal computers are Microsoft Windows, macOS, and 

In [6]:
tokenized_corpus = [nltk.word_tokenize(passage) for passage in corpus]
bm25 = BM25Okapi(tokenized_corpus)

doc_scores = bm25.get_scores(query_words)
max_score_index = np.argmax(doc_scores)
print(doc_scores)
print(max_score_index)
print(corpus[max_score_index])

[0.8355942 0.        0.       ]
0
An operating system is the most important software that runs on a computer. It manages the computer's memory and processes, as well as all of its software and hardware. It also allows you to communicate with the computer without knowing how to speak the computer's language. Without an operating system, a computer is useless.
Your computer's operating system (OS) manages all of the software and hardware on the computer. Most of the time, there are several different computer programs running at the same time, and they all need to access your computer's central processing unit (CPU), memory, and storage. The operating system coordinates all of this to make sure each program gets what it needs.
Operating systems usually come pre-loaded on any computer you buy. Most people use the operating system that comes with their computer, but it's possible to upgrade or even change operating systems. The three most common operating systems for personal computers are 

Answer Retrieving

In [7]:
from transformers import pipeline

In [8]:
nlp = pipeline("question-answering")

In [9]:
OS_original_text = ''
with open("OS_extended.txt") as file:
    OS_original_text = file.read()
print(OS_original_text)

print(nlp(question="What are the different OS for personal computers?", context=corpus[max_score_index]))
print(nlp(question="What does operating system coordinates?", context=corpus[max_score_index]))
print(nlp(question="What is necessary for a programs to run?",
          context=corpus[max_score_index]))

An operating system is the most important software that runs on a computer. It manages the computer's memory and processes, as well as all of its software and hardware. It also allows you to communicate with the computer without knowing how to speak the computer's language. Without an operating system, a computer is useless.
Your computer's operating system (OS) manages all of the software and hardware on the computer. Most of the time, there are several different computer programs running at the same time, and they all need to access your computer's central processing unit (CPU), memory, and storage. The operating system coordinates all of this to make sure each program gets what it needs.
Operating systems usually come pre-loaded on any computer you buy. Most people use the operating system that comes with their computer, but it's possible to upgrade or even change operating systems. The three most common operating systems for personal computers are Microsoft Windows, macOS, and Linu

Using Model and tokenizer

In [10]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

In [ ]:
#!pip install transformers sentencepiece

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad", return_dict=False)

In [12]:
questions = [
    "What are the different OS for personal computers?",
    "What does operating system coordinates?",
    "What is necessary for a programs to run?",
]

for question in questions:
    inputs = tokenizer.encode_plus(question, OS_original_text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    print(input_ids)

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs)

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

[101, 2054, 2024, 1996, 2367, 9808, 2005, 3167, 7588, 1029, 102, 2019, 4082, 2291, 2003, 1996, 2087, 2590, 4007, 2008, 3216, 2006, 1037, 3274, 1012, 2009, 9020, 1996, 3274, 1005, 1055, 3638, 1998, 6194, 1010, 2004, 2092, 2004, 2035, 1997, 2049, 4007, 1998, 8051, 1012, 2009, 2036, 4473, 2017, 2000, 10639, 2007, 1996, 3274, 2302, 4209, 2129, 2000, 3713, 1996, 3274, 1005, 1055, 2653, 1012, 2302, 2019, 4082, 2291, 1010, 1037, 3274, 2003, 11809, 1012, 2115, 3274, 1005, 1055, 4082, 2291, 1006, 9808, 1007, 9020, 2035, 1997, 1996, 4007, 1998, 8051, 2006, 1996, 3274, 1012, 2087, 1997, 1996, 2051, 1010, 2045, 2024, 2195, 2367, 3274, 3454, 2770, 2012, 1996, 2168, 2051, 1010, 1998, 2027, 2035, 2342, 2000, 3229, 2115, 3274, 1005, 1055, 2430, 6364, 3131, 1006, 17368, 1007, 1010, 3638, 1010, 1998, 5527, 1012, 1996, 4082, 2291, 12093, 2035, 1997, 2023, 2000, 2191, 2469, 2169, 2565, 4152, 2054, 2009, 3791, 1012, 4082, 3001, 2788, 2272, 3653, 1011, 8209, 2006, 2151, 3274, 2017, 4965, 1012, 2087, 2111, 2